<a href="https://colab.research.google.com/github/KalharaBatangala/Brain-tumor-detection/blob/main/Brain_Tumor_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import keras

from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from sklearn.metrics import accuracy_score

import ipywidgets as widgets
import io
from PIL import Image
import tqdm
from sklearn.model_selection import train_test_split
import cv2
from sklearn.utils import shuffle
import tensorflow as tf
import os


In [ ]:
X_train = []
Y_train = []
image_size = 150
labels = ['glioma', 'meningioma', 'pituitary', 'notumor']

for i in labels:
  folderPath = os.path.join('./drive/MyDrive/DatasetMRI/Training/', i)
  for j in os.listdir(folderPath):
    img = cv2.imread(os.path.join(folderPath, j))
    img = cv2.resize(img, (image_size, image_size))
    X_train.append(img)
    Y_train.append(i)

X_train = np.array(X_train)
Y_train = np.array(Y_train)
print('Completed successfully')

Completed successfully


In [ ]:
X_train, Y_train = shuffle(X_train, Y_train, random_state=101)

In [ ]:
X_train[10]

150

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Train test split

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Load a sample dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Define a simple model
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5)

# Evaluate the model
model.evaluate(x_test, y_test)


Num GPUs Available:  1
11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/5
1875/1875 [==============================] - 9s 3ms/step - loss: 0.2940 - accuracy: 0.9151
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1411 - accuracy: 0.9577
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1054 - accuracy: 0.9682
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0896 - accuracy: 0.9721
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0799 - accuracy: 0.9757


[0.07992273569107056, 0.9757000207901001]